### Ньяти Каелиле БВТ2201 Лаб 1

In [26]:
pip install scikit-learn gensim nltk numpy


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### TF-IDF (Term Frequency - Inverse Document Frequency)

In [1]:
#TF-IDF = TF * IDF (formula)

import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer

# Load dataset (first 100 rows)
csv_path = "archive/train.csv"
df = pd.read_csv(csv_path).head(100)

# Preprocessing functions
def clean_data(text):
    text = re.sub(r'[^ \nA-Za-z0-9À-ÖØ-öø-ÿ/]+', '', text)  # Keep letters & numbers
    text = re.sub(r'[\\/×\^\]\[÷]', '', text)  # Remove extra symbols
    return text.lower()  # Convert to lowercase

# Apply preprocessing
df["Description"] = df["Description"].astype(str)
df["Description"] = df["Description"].apply(clean_data)

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the "Description" column
tfidf_matrix = vectorizer.fit_transform(df["Description"])

# Convert to array for better visualization
tfidf_array = tfidf_matrix.toarray()

# Print results
print("TF-IDF Matrix:")
print(tfidf_array)
print("\nVocabulary:", vectorizer.get_feature_names_out())


TF-IDF Matrix:
[[0.         0.         0.         ... 0.18678747 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.02164468 ... 0.07405656 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.08903058 ... 0.         0.         0.        ]]

Vocabulary: ['aaa' 'abilities' 'able' ... 'youre' 'yourself' 'zoo']


### Word2Vec

#### training

In [4]:
import pandas as pd
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import multiprocessing


csv_path = "archive/train.csv"

df = pd.read_csv(csv_path)
df = df.head(1000)  # Limit dataset to 100 rows

def clean_data(text):
    text = re.sub(r'[^ \nA-Za-z0-9À-ÖØ-öø-ÿ/]+', '', text)
    text = re.sub(r'[\\/×\^\]\[÷]', '', text)
    return text
def change_lower(text):
    text = text.lower()
    return text

stopwords_list = stopwords.words("english")
def remover(text):
    text_tokens = text.split(" ")
    final_list = [word for word in text_tokens if not word in stopwords_list]
    text = ' '.join(final_list)
    return text

def get_w2vdf(df):
    w2v_df = pd.DataFrame(df["Description"]).values.tolist()
    for i in range(len(w2v_df)):
        w2v_df[i] = w2v_df[i][0].split(" ")
    return w2v_df

#sg=1 {skipgram algorithm is used: target word is input, and neighboring words are output}
def train_w2v(w2v_df):
    cores = multiprocessing.cpu_count()
    w2v_model = Word2Vec(min_count=4,
                         window=4,
                         vector_size=300, 
                         alpha=0.03, 
                         min_alpha=0.0007, 
                         sg = 1,
                         workers=cores-1)
    
    w2v_model.build_vocab(w2v_df, progress_per=10000)
    w2v_model.train(w2v_df, total_examples=w2v_model.corpus_count, epochs=100, report_delay=1)
    return w2v_model

print(df.columns)


df["Description"] = df["Description"].astype(str)
df["Description"] = df["Description"].apply(change_lower)
df["Description"] = df["Description"].apply(clean_data)
df["Description"] = df["Description"].apply(remover)

w2v_df = get_w2vdf(df)
w2v_model = train_w2v(w2v_df)


# Save the model 
w2v_model.save("word2vec.model")


Index(['User_ID', 'Description', 'Browser_Used', 'Device_Used', 'Is_Response'], dtype='object')


### testing

In [6]:
# Load the model
model = Word2Vec.load("word2vec.model")

# Perform word vector operations
vector = model.wv['machine']  # Get vector for a word
similar_words = model.wv.most_similar('machine')  # Find similar words
result = model.wv.most_similar(positive=['boyfriend', 'wife'], negative=['husband'])


print("Vector for 'machine':", vector)
print("Words similar to 'machine':", similar_words)
print("boyfriend - husband + wife ≈", result)
print(w2v_model.wv["great"].shape)

Vector for 'machine': [-2.51948893e-01  5.07174850e-01 -2.41980433e-01  7.04491854e-01
  1.85922578e-01 -3.98904413e-01  2.55203873e-01 -5.42556107e-01
  2.39533916e-01  4.78349239e-01 -3.37609619e-01  6.02217138e-01
 -2.25125790e-01  7.25654187e-03  7.23421812e-01 -3.49665225e-01
  7.06573009e-01 -9.04319361e-02 -6.11862719e-01  1.01483598e-01
 -6.72746658e-01  6.36112928e-01  2.05288514e-01  4.27977055e-01
  2.96130151e-01 -8.47974140e-03 -4.74351421e-02  1.83336020e-01
 -4.50801104e-01 -6.69171661e-03 -2.68301755e-01 -4.60748851e-01
  7.79018342e-01 -4.77325529e-01  3.19934636e-02  3.36215734e-01
  2.64936090e-01  1.29701719e-01  3.39320332e-01 -1.88799381e-01
 -1.12151548e-01  7.05033168e-03 -7.85353124e-01 -2.98958570e-02
 -2.15646893e-01  1.18268929e-01  6.80338144e-01 -3.13293338e-01
 -2.37163574e-01 -2.10135862e-01  1.45742968e-01 -4.79484528e-01
 -1.54954195e-01  5.10323286e-01  2.21388936e-02 -2.78583556e-01
 -9.21114441e-03 -1.64979964e-01 -3.55524383e-02 -1.55242920e-01
 -3

In [8]:
# Load test dataset
test_csv_path = "archive/test.csv"
df_test = pd.read_csv(test_csv_path)

# Apply preprocessing to test data
df_test["Description"] = df_test["Description"].astype(str)
df_test["Description"] = df_test["Description"].apply(change_lower)
df_test["Description"] = df_test["Description"].apply(clean_data)
df_test["Description"] = df_test["Description"].apply(remover)

# Convert test data into word lists
w2v_test_data = get_w2vdf(df_test)

# Load the trained Word2Vec model
model = Word2Vec.load("word2vec.model")

# Example: Get word vectors from test data
for sentence in w2v_test_data[:5]:  # Show vectors for first 5 rows
    sentence_vectors = [model.wv[word] for word in sentence if word in model.wv]
    print(f"Sentence: {' '.join(sentence)}")
    print(f"Word vectors: {sentence_vectors}\n")


result = model.wv.most_similar(positive=['boyfriend', 'wife'], negative=['husband'])
print("boyfriend - husband + wife ≈", result)

# Example: Find similar words from test data
if "machine" in model.wv:
    similar_words = model.wv.most_similar("machine")
    print(f"Words similar to 'machine': {similar_words}")
else:
    print("'machine' not in vocabulary.")


    

Sentence: looking motel close proximity tv taping dr phil show chose dunes sunset blvd west hollywood although property displayed aaa emblem certainly left lot desired chips  scrapes bottom door frame bathroom lotion containers half fullapparently replaced housekeeping needed early wakeup call couldnt use clock radio alarm wasnt radio room tv channel listing remote tv menu making viewing chorethe tv remote returned checkingout place served purpose place revisit
Word vectors: [array([ 6.15962207e-01,  8.04592818e-02, -3.60216856e-01, -1.66775122e-01,
        1.32730510e-03, -1.73207089e-01,  6.84187859e-02,  3.42356294e-01,
        2.67854214e-01,  4.98764098e-01,  4.81809020e-01, -4.42093194e-01,
        7.41234839e-01, -7.54235148e-01, -6.65862560e-02,  1.86927337e-02,
       -2.22418010e-01,  2.50413865e-01,  5.58642149e-01,  1.63388744e-01,
        2.02110976e-01, -6.83738172e-01, -5.81881106e-01,  2.31215104e-01,
        1.25248395e-02, -1.93591624e-01,  2.30699345e-01,  5.30668497